In [ ]:
import pandas
from tqdm import tqdm

xls = pandas.ExcelFile("/srv/kkotochigov/kimberly/tax.xlsx")

attributes_sheet = xls.parse(0)[['ID','Type','Description']]

fails = 0
dataframes_to_concat = []
for sheet_num, sheet_name in tqdm(enumerate(xls.sheet_names), total=len(xls.sheet_names)):
    if sheet_num<2: 
        continue
    parsed_data = xls.parse(sheet_name=sheet_name)
    if ('ID' in parsed_data.columns) & ('Description' in parsed_data.columns):
        parsed_data['group_name'] = sheet_name
        parsed_data = parsed_data[['group_name','ID', 'Description']]
        dataframes_to_concat.append(parsed_data)
    else:
        fails += 1
# attributes_sheet
values_sheet = pandas.concat(dataframes_to_concat)

values_sheet.group_name.value_counts()

In [ ]:
df = pandas.merge(attributes_sheet, values_sheet, left_on='Type', right_on='group_name', how='left')
df = df.drop(columns=['Type','group_name'])
df.columns = ['group_id','group_desc','attr_id','attr_desc']
df.to_csv("taxonomy.csv",sep=',', index=False)